## 外れ値が多かったり, Loyalityに特徴があるCity, State, Subsectorを特徴にする

In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [2]:
hist = utils.read_df_pkl('../input/historical_transactions0*')
new = utils.read_df_pkl('../input/new*0*')
df = pd.concat([hist, new], axis=0)

100%|██████████| 3/3 [00:01<00:00,  2.12it/s]


### City_id

In [4]:
#========================================================================
# City_id
#========================================================================

df['cnt'] = 1
df_city = df.groupby([key, 'city_id'])['cnt'].sum().reset_index()
df_rec = df.groupby([key])['cnt'].sum().reset_index()

main = df_city.merge(df_rec, how='inner', on=key)
del df_city, df_rec
gc.collect()
display(main.head(20))

df['authorized_flag'] = df['authorized_flag'].map(lambda x: 0 if x=='N' else 1)
auth_mean = df.groupby('city_id')['authorized_flag'].mean()
auth_mean.head()

,card_id,city_id,cnt_x,cnt_y
0,C_ID_00007093c1,-1,28,151
1,C_ID_00007093c1,69,1,151
2,C_ID_00007093c1,76,21,151
3,C_ID_00007093c1,244,100,151
4,C_ID_00007093c1,333,1,151
5,C_ID_0001238066,-1,11,149
6,C_ID_0001238066,19,2,149
7,C_ID_0001238066,28,1,149
8,C_ID_0001238066,53,4,149
9,C_ID_0001238066,69,18,149


SystemExit: 

### main_cityのidとcityのauth_mean, そのcityでの購買割合を特徴にする

In [15]:
auth_city = main_city.map(auth_mean)
auth_city.name = 'city_auth_mean'

main_city.name = 'main_city_id'
main_city_ratio.name = 'main_city_ratio'

result = pd.concat([auth_city, main_city, main_city_ratio], axis=1)

result.reset_index(inplace=True)
result.head()

,city_auth_mean,main_city_id,main_city_ratio
card_id,,,
C_ID_00007093c1,0.901376,244,0.662252
C_ID_0001238066,0.945632,314,0.315436
C_ID_0001506ef0,0.927034,137,0.955882
C_ID_0001793786,0.924189,179,0.461538
C_ID_000183fdda,0.937717,161,0.858065


In [17]:
feat_no = '123_mai_'
base = utils.read_df_pkl('../input/base*')
base_train = base[~base[target].isnull()].reset_index(drop=True)
base_test = base[base[target].isnull()].reset_index(drop=True)

df_train = base_train.merge(result, how='left', on=key)
df_test = base_test.merge(result, how='left', on=key)

for col in df_train.columns:
    if col in ignore_list:continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{feat_no}train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{feat_no}test_{col}@', obj=df_test[col].values)

100%|██████████| 3/3 [00:00<00:00, 48.58it/s]


### 124 historicalとnewでのmain cityの違いを特徴にする

In [19]:
#========================================================================
# City_id
#========================================================================

hist['cnt'] = 1
hist_city = hist.groupby([key, 'city_id'])['cnt'].sum().reset_index()
hist_rec = hist.groupby([key])['cnt'].sum().reset_index()

new['cnt'] = 1
new_city = new.groupby([key, 'city_id'])['cnt'].sum().reset_index()
new_rec = new.groupby([key])['cnt'].sum().reset_index()

hist_main = hist_city.merge(hist_rec, how='inner', on=key)
new_main = new_city.merge(new_rec, how='inner', on=key)
del hist_city, hist_rec, new_city, new_rec
gc.collect()
display(hist_main.head(10))

hist_main['city_ratio'] = hist_main['cnt_x'] / hist_main['cnt_y']
hist_main_city = hist_main.set_index('city_id').groupby(key)['city_ratio'].idxmax()
hist_main_city_ratio = hist_main.groupby(key)['city_ratio'].max()

new_main['city_ratio'] = new_main['cnt_x'] / new_main['cnt_y']
new_main_city = new_main.set_index('city_id').groupby(key)['city_ratio'].idxmax()
new_main_city_ratio = new_main.groupby(key)['city_ratio'].max()

In [21]:
hist_main_city.name = 'hist_main_city_id'
hist_main_city_ratio.name = 'hist_main_city_ratio'

new_main_city.name = 'new_main_city_id'
new_main_city_ratio.name = 'new_main_city_ratio'

result = pd.concat([hist_main_city, hist_main_city_ratio, new_main_city, new_main_city_ratio], axis=1)

result.reset_index(inplace=True)
result.head()

,index,hist_main_city_id,hist_main_city_ratio,new_main_city_id,new_main_city_ratio
0,C_ID_00007093c1,244,0.671141,69.0,0.500000
1,C_ID_0001238066,314,0.325203,314.0,0.269231
2,C_ID_0001506ef0,137,0.954545,137.0,1.000000
3,C_ID_0001793786,179,0.518519,69.0,0.483871
4,C_ID_000183fdda,161,0.854167,161.0,0.909091


In [23]:
result.rename(columns={'index':key}, inplace=True)

result['diff_hist_new_main_city_ratio'] = result['hist_main_city_ratio'] - result['new_main_city_ratio']
result['change_main_city_flg'] = (result['hist_main_city_id'] == result['new_main_city_id']) * -1 + 1
result.head()

,card_id,hist_main_city_id,hist_main_city_ratio,new_main_city_id,new_main_city_ratio,diff_hist_new_main_city_ratio,change_main_city_flg
0,C_ID_00007093c1,244,0.671141,69.0,0.500000,0.171141,1
1,C_ID_0001238066,314,0.325203,314.0,0.269231,0.055972,0
2,C_ID_0001506ef0,137,0.954545,137.0,1.000000,-0.045455,0
3,C_ID_0001793786,179,0.518519,69.0,0.483871,0.034648,1
4,C_ID_000183fdda,161,0.854167,161.0,0.909091,-0.054924,0


In [25]:
feat_no = '123_mai_'
base = utils.read_df_pkl('../input/base*')
base_train = base[~base[target].isnull()].reset_index(drop=True)
base_test = base[base[target].isnull()].reset_index(drop=True)

df_train = base_train.merge(result, how='left', on=key)
df_test = base_test.merge(result, how='left', on=key)

for col in df_train.columns:
    if col in ignore_list:continue
    if not(col.count('diff')) and not(col.count('change')) :continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{feat_no}train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{feat_no}test_{col}@', obj=df_test[col].values)

100%|██████████| 3/3 [00:00<00:00, 86.91it/s]


### State_id

In [40]:
#========================================================================
# State_id
feat_col = 'state_id'
f4 = pd.read_csv('../input/state_ratio_out_cnt_id.csv')

# 1
l1 = f4[f4.ratio_out<0.005][feat_col].values.tolist()

# feat_list = list(set(l1 + l2 + l3 + l4 + l5 + l6))
feat_list = list(set(l5 + l6))
#========================================================================

# Squeeze City
df_m = df[[feat_col, 'card_id']].copy()

# One Hot
df_m['value'] = 1
ohe = df_m.pivot_table(index=key, columns=feat_col, values='value', aggfunc='sum').fillna(0).astype('int')

ohe.head()

   state_id  cnt_id  ratio_out  state_cnt_id
0        -1  103783   0.006190        167432
1         1    5911   0.006480          9509
2         2    7941   0.006948         12864
3         3   10209   0.005948         16572
4         4   15750   0.006825         25424


SystemExit: 

In [37]:
for col in ohe.columns:
    ohe.rename(columns={col:f"{feat_col}_{col}"}, inplace=True)
    
card_row = df_m.groupby(key).size()
card_row.name = 'index'
ohe = ohe.join(card_row)

for col in ohe.columns:
    if col.count(feat_col):
        ohe[col] = ohe[col]/ohe['index']
        
ohe.head()

,state_id_-1,state_id_1,state_id_2,state_id_3,state_id_4,state_id_5,state_id_6,state_id_7,state_id_8,state_id_9,state_id_10,state_id_11,state_id_12,state_id_13,state_id_14,state_id_15,state_id_16,state_id_17,state_id_18,state_id_19,state_id_20,state_id_21,state_id_22,state_id_23,state_id_24,index
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,0.185430,0.0,0.801325,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.006623,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.006623,0.0,0.0,0.000000,151
C_ID_0001238066,0.073826,0.0,0.000000,0.000000,0.0,0.006711,0.0,0.000000,0.0,0.751678,0.0,0.000000,0.0,0.0,0.0,0.020134,0.0,0.0,0.000000,0.000000,0.14094,0.006711,0.0,0.0,0.000000,149
C_ID_0001506ef0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.029412,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.970588,0.00000,0.000000,0.0,0.0,0.000000,68
C_ID_0001793786,0.469636,0.0,0.000000,0.000000,0.0,0.004049,0.0,0.000000,0.0,0.105263,0.0,0.080972,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.340081,247
C_ID_000183fdda,0.025806,0.0,0.000000,0.890323,0.0,0.006452,0.0,0.019355,0.0,0.045161,0.0,0.006452,0.0,0.0,0.0,0.000000,0.0,0.0,0.006452,0.000000,0.00000,0.000000,0.0,0.0,0.000000,155


In [38]:
base = utils.read_df_pkl('../input/base*')
base_train = base[~base[target].isnull()].reset_index(drop=True)
base_test = base[base[target].isnull()].reset_index(drop=True)

df_train = base_train.merge(ohe, how='left', on=key)
df_test = base_test.merge(ohe, how='left', on=key)

for col in df_train.columns:
    if col in ignore_list or col.count('index'):continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/115_out_train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/115_out_test_{col}@', obj=df_test[col].values)

100%|██████████| 3/3 [00:00<00:00, 33.54it/s]
